In [1]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer

import re
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

 Using Facebook Dataset to find which category(economy, microsoft, obama, palestine) the news article belong 

In [2]:
fb_economy = pd.read_csv('Facebook_Economy.csv')

In [3]:
fb_economy.isnull().any().any()

False

In [4]:
fb_economy.shape

(29928, 145)

In [5]:
fb_microsoft = pd.read_csv('Facebook_Microsoft.csv')

In [6]:
fb_microsoft.isnull().any().any()

False

In [7]:
fb_microsoft.shape

(18531, 145)

In [8]:
fb_obama = pd.read_csv('Facebook_Obama.csv')

In [9]:
fb_obama.isnull().any().any()

False

In [10]:
fb_obama.shape

(27015, 145)

In [11]:
fb_palestine = pd.read_csv('Facebook_Palestine.csv')

In [12]:
fb_palestine.isnull().any().any()

False

In [13]:
fb_palestine.shape

(7687, 145)

In [14]:
fb_data = pd.concat([fb_economy, fb_microsoft,
                    fb_obama, fb_palestine], ignore_index = True)

In [15]:
fb_data.head()

,IDLink,TS1,TS2,TS3,TS4,TS5,TS6,TS7,TS8,TS9,...,TS135,TS136,TS137,TS138,TS139,TS140,TS141,TS142,TS143,TS144
0,1.0,-1,-1,-1,-1,-1,-1,-1,-1,7,...,13,13,13,13,13,13,13,13,13,13
1,2.0,-1,-1,-1,-1,-1,-1,-1,-1,3,...,42,42,42,42,42,42,42,42,42,42
2,3.0,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,98,98,98,98,98,98,98,98,98,98
3,4.0,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,7,7,7,7,7,7,7,7,7,7
4,5.0,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,34,34,34,34,34,34,34,34,34,34


In [16]:
fb_data.shape

(83161, 145)

In [17]:
data = pd.read_csv('News_Final.csv')

In [18]:
data.head()

,IDLink,Title,Headline,Source,Topic,PublishDate,SentimentTitle,SentimentHeadline,Facebook,GooglePlus,LinkedIn
0,99248.0,Obama Lays Wreath at Arlington National Cemetery,Obama Lays Wreath at Arlington National Cemete...,USA TODAY,obama,2002-04-02 00:00:00,0.000000,-0.053300,-1,-1,-1
1,10423.0,A Look at the Health of the Chinese Economy,"Tim Haywood, investment director business-unit...",Bloomberg,economy,2008-09-20 00:00:00,0.208333,-0.156386,-1,-1,-1
2,18828.0,Nouriel Roubini: Global Economy Not Back to 2008,"Nouriel Roubini, NYU professor and chairman at...",Bloomberg,economy,2012-01-28 00:00:00,-0.425210,0.139754,-1,-1,-1
3,27788.0,Finland GDP Expands In Q4,Finland's economy expanded marginally in the t...,RTT News,economy,2015-03-01 00:06:00,0.000000,0.026064,-1,-1,-1
4,27789.0,"Tourism, govt spending buoys Thai economy in J...",Tourism and public spending continued to boost...,The Nation - Thailand&#39;s English news,economy,2015-03-01 00:11:00,0.000000,0.141084,-1,-1,-1


In [19]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93239 entries, 0 to 93238
Data columns (total 11 columns):
IDLink               93239 non-null float64
Title                93239 non-null object
Headline             93224 non-null object
Source               92960 non-null object
Topic                93239 non-null object
PublishDate          93239 non-null object
SentimentTitle       93239 non-null float64
SentimentHeadline    93239 non-null float64
Facebook             93239 non-null int64
GooglePlus           93239 non-null int64
LinkedIn             93239 non-null int64
dtypes: float64(3), int64(3), object(5)
memory usage: 7.8+ MB


In [20]:
facebook = pd.merge(data, fb_data, on = "IDLink")

In [21]:
facebook.head()

,IDLink,Title,Headline,Source,Topic,PublishDate,SentimentTitle,SentimentHeadline,Facebook,GooglePlus,...,TS135,TS136,TS137,TS138,TS139,TS140,TS141,TS142,TS143,TS144
0,80690.0,"Monday, 29 Feb 2016","RAMALLAH, February 25, 2016 (WAFA) - Palestine...",NaN,palestine,2016-02-28 14:03:00,0.000000,-0.005906,0,0,...,0,0,0,0,0,0,0,0,0,0
1,80690.0,"Monday, 29 Feb 2016","RAMALLAH, February 25, 2016 (WAFA) - Palestine...",NaN,palestine,2016-02-28 14:03:00,0.000000,-0.005906,0,0,...,0,0,0,0,0,0,0,0,0,0
2,81052.0,"Monday, 29 Feb 2016","RAMALLAH, February 29, 2016 (WAFA) - The Gover...",NaN,palestine,2016-03-01 09:29:00,0.000000,0.048546,0,0,...,0,0,0,0,0,0,0,0,0,0
3,81052.0,"Monday, 29 Feb 2016","RAMALLAH, February 29, 2016 (WAFA) - The Gover...",NaN,palestine,2016-03-01 09:29:00,0.000000,0.048546,0,0,...,0,0,0,0,0,0,0,0,0,0
4,80994.0,"Tuesday, 1 Mar 2016","RAMALLAH, February 29, 2016 (WAFA) - The Gover...",NaN,palestine,2016-03-01 00:15:00,-0.243068,0.048546,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
facebook.shape

(83164, 155)

In [23]:
facebook['Headline'].isnull().sum()

12

In [24]:
facebook['Topic'].isnull().sum()

0

In [25]:
facebook = facebook.dropna()

In [26]:
facebook.shape

(82929, 155)

In [27]:
facebook.head()

,IDLink,Title,Headline,Source,Topic,PublishDate,SentimentTitle,SentimentHeadline,Facebook,GooglePlus,...,TS135,TS136,TS137,TS138,TS139,TS140,TS141,TS142,TS143,TS144
6,31.0,Can the AK party save Turkey's flagging economy?,"According to the IMF and World Bank, the once ...",Aljazeera.com,economy,2015-11-08 00:00:00,0.000000,0.116946,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
7,50.0,China Economic Slowdown Restricts Reform Effor...,China is delaying plans to loosen controls on ...,International Business Times,economy,2015-11-08 00:00:00,0.178986,0.091225,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
8,51.0,"Diwali sales, analysts and companies indicate ...",It's also a time when virtually every other ma...,Economic Times,economy,2015-11-08 00:00:00,0.000000,-0.099938,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
9,57.0,China Delays Economic Liberalization,"BEIJING""""""The closed-door meeting of some of C...",Wall Street Journal,economy,2015-11-08 00:00:00,0.250000,0.155656,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
10,59.0,Fast internet for all will be 'shot in the arm...,The Confederation of British Industry (CBI) ha...,Telegraph.co.uk,economy,2015-11-08 00:00:00,0.200446,0.121139,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [28]:
facebook.shape

(82929, 155)

In [29]:
facebook.reset_index(inplace = True) 

In [30]:
facebook.tail()

,index,IDLink,Title,Headline,Source,Topic,PublishDate,SentimentTitle,SentimentHeadline,Facebook,...,TS135,TS136,TS137,TS138,TS139,TS140,TS141,TS142,TS143,TS144
82924,83159,104788.0,"Despite Obama's new executive order, US drone ...","In 2013, President Obama promised to bring gre...",Washington Post,obama,2016-07-07 14:05:15,-0.229171,-0.163486,9,...,9,9,9,9,9,9,9,9,9,9
82925,83160,104797.0,Obama's final fight,That's what US President Barack Obama confront...,gulfnews.com,obama,2016-07-07 14:20:15,-0.505181,0.098752,0,...,0,0,0,0,0,0,0,0,0,0
82926,83161,104799.0,Tournoi de foot solidaire pour la Palestine,Dans le cadre de la campagne annuelle de solid...,L'Humanité,palestine,2016-07-07 15:20:27,-0.047246,-0.106600,4,...,4,4,4,4,4,4,4,4,4,4
82927,83162,104793.0,Palestinian Government Uses Foreign Aid To Pay...,The Palestinian government spends nearly $140 ...,Daily Caller,palestine,2016-07-07 15:38:26,0.291667,-0.139754,5,...,5,5,5,5,5,5,5,5,5,5
82928,83163,104794.0,Palestine Youth Orchestra prepares for first U...,Palestine Youth Orchestra prepares for first U...,Ahram Online,palestine,2016-07-07 15:59:22,0.121534,0.092313,0,...,0,0,0,0,0,0,0,0,0,0


In [31]:
facebook.drop(['index'], axis =1, inplace = True)

In [31]:
facebook.head()

,IDLink,Title,Headline,Source,Topic,PublishDate,SentimentTitle,SentimentHeadline,Facebook,GooglePlus,...,TS135,TS136,TS137,TS138,TS139,TS140,TS141,TS142,TS143,TS144
0,31.0,Can the AK party save Turkey's flagging economy?,"According to the IMF and World Bank, the once ...",Aljazeera.com,economy,2015-11-08 00:00:00,0.000000,0.116946,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,50.0,China Economic Slowdown Restricts Reform Effor...,China is delaying plans to loosen controls on ...,International Business Times,economy,2015-11-08 00:00:00,0.178986,0.091225,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,51.0,"Diwali sales, analysts and companies indicate ...",It's also a time when virtually every other ma...,Economic Times,economy,2015-11-08 00:00:00,0.000000,-0.099938,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3,57.0,China Delays Economic Liberalization,"BEIJING""""""The closed-door meeting of some of C...",Wall Street Journal,economy,2015-11-08 00:00:00,0.250000,0.155656,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
4,59.0,Fast internet for all will be 'shot in the arm...,The Confederation of British Industry (CBI) ha...,Telegraph.co.uk,economy,2015-11-08 00:00:00,0.200446,0.121139,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [32]:
facebook.isnull().any().any()

False

In [33]:
facebook.shape

(82929, 155)

Using the headline as feature and Topic as label

In [34]:
corpus_headline = [ ]

In [35]:
facebook['Headline'][0]

'According to the IMF and World Bank, the once booming economy that grew 9.2 percent in 2010 is now forecast to grow by just 3 percent this'

In [36]:
for i in range(82929):
    headline = re.sub('[^a-zA-Z]+', ' ', facebook['Headline'][i])
    headline = re.sub('\s+', ' ', headline).lower()
    
    stemmer = PorterStemmer()
    headline = [stemmer.stem(w) for w in word_tokenize(headline) if w not in stopwords.words('english')]
    headline = ' '.join(headline)
    
    corpus_headline.append(headline)


In [37]:
len(corpus_headline)

82929

Applying label Encoding on topics column of data

In [38]:
lb = LabelEncoder()
y =lb.fit_transform(facebook['Topic'])

In [39]:
cv = CountVectorizer(max_df = 0.90, min_df = 2, max_features = 5000)

Reduced the Features 30587 from to 5000 as memory is insufficient to handle shape of (82929,30587) sparse matrix

In [40]:
x = cv.fit_transform(corpus_headline).toarray()

In [41]:
x.shape

(82929, 5000)

In [42]:
x_train = x[:20000]
x_test = x[20000:40000]
y_train = y[:20000]
y_test = y[20000:40000]

In [43]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((20000, 5000), (20000, 5000), (20000,), (20000,))

Applying different classifiers to predict the category of news article

Using Gaussian Naive Bayes

In [44]:
nb = GaussianNB()
nb.fit(x_train, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [45]:
y_pred_nb = nb.predict(x_test)

In [46]:
f1_score_nb = f1_score(y_test, y_pred_nb, average = 'weighted')
score_nb = accuracy_score(y_test, y_pred_nb)
matrix_nb = confusion_matrix(y_test, y_pred_nb)
print('F1 Score-:', f1_score_nb)
print('Accuracy Score-:', score_nb)
print('Confusion Matrix-:', matrix_nb)

F1 Score-: 0.662443702010759
Accuracy Score-: 0.6583
Confusion Matrix-: [[4141 1416 1109  802]
 [ 282 3848  260   39]
 [ 866   57 3815 1543]
 [ 144   60  256 1362]]


In [47]:
y_pred_nb

array([1, 3, 2, ..., 1, 1, 1])

Using Logistic Regression

In [48]:
lr = LogisticRegression()
lr.fit(x_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [49]:
y_pred_lr = lr.predict(x_test)

In [50]:
f1_score_lr = f1_score(y_test, y_pred_lr, average = 'weighted')
score_lr = accuracy_score(y_test, y_pred_lr)
matrix_lr = confusion_matrix(y_test, y_pred_lr)
print('F1 Score-:', f1_score_lr)
print('Accuracy Score-:', score_lr)
print('Confusion Matrix-:', matrix_lr)

F1 Score-: 0.9647376698235428
Accuracy Score-: 0.96475
Confusion Matrix-: [[7305   25  130    8]
 [ 105 4282   35    7]
 [ 210   13 6042   16]
 [  96   17   43 1666]]


In [52]:
y_pred_lr[0:50]

array([1, 2, 2, 2, 0, 2, 1, 2, 2, 0, 1, 1, 1, 1, 0, 3, 2, 2, 0, 2, 2, 1,
       0, 1, 2, 2, 3, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 2,
       1, 2, 2, 2, 1, 1])

The headlines are now classified in to topics('economics', 'microsoft', 'palestine', 'obama') in y_pred_nb.
Using y_pred_nb as the input and TS144(Final level of popularity after 2 days upon publication) as the output
With this and using Regression we can predict the popularity of the news headline

In [98]:
popularity = facebook['TS144'].iloc[20000:40000]
popularity.shape

(20000,)

In [99]:
popularity

20000       0
20001     141
20002      72
20003     896
20004      21
         ... 
39995       3
39996       0
39997      17
39998    1097
39999       3
Name: TS144, Length: 20000, dtype: int64

In [100]:
features = {'News_Categories':y_pred_nb, 'Popularity':popularity}

In [101]:
df = pd.DataFrame(features)

In [102]:
df.head()

,News_Categories,Popularity
20000,1,0
20001,3,141
20002,2,72
20003,3,896
20004,1,21


In [103]:
df.shape

(20000, 2)

importing the library for linear regression

In [104]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [105]:
X = df.iloc[:, 0:1].values
Y = df.iloc[:,-1:].values

In [106]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0)

In [107]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((16000, 1), (4000, 1), (16000, 1), (4000, 1))

In [108]:
classifier = LinearRegression()
classifier.fit(x_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [109]:
y_pred_linear = classifier.predict(x_test)

In [110]:
mse = mean_squared_error(y_test, y_pred_linear)
rmse = np.sqrt(mse)
print('MSE:-', mse)
print('RMSE:-', rmse)

MSE:- 508525.7398391713
RMSE:- 713.1099072647717


Using Logistic Regression predictions

In [111]:
features_lr = {'News_Categories':y_pred_lr, 'Popularity':popularity}

In [112]:
df_lr = pd.DataFrame(features_lr)

In [113]:
df_lr.head()

,News_Categories,Popularity
20000,1,0
20001,2,141
20002,2,72
20003,2,896
20004,0,21


In [114]:
df_lr.shape

(20000, 2)

In [115]:
X_lr = df.iloc[:, 0:1].values
Y_lr = df.iloc[:,-1:].values

In [116]:
x_train, x_test, y_train, y_test = train_test_split(X_lr, Y_lr, test_size = 0.2, random_state = 0)

In [117]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((16000, 1), (4000, 1), (16000, 1), (4000, 1))

In [122]:
classifier_lr = LinearRegression()
classifier_lr.fit(x_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [123]:
y_pred_lr = classifier_lr.predict(x_test)

In [124]:
mse = mean_squared_error(y_test, y_pred_lr)
rmse = np.sqrt(mse)
print('MSE:-', mse)
print('RMSE:-', rmse)

MSE:- 508525.7398391713
RMSE:- 713.1099072647717
